## Introduction
DNS is a hierarchical distributed database that contains mappings of DNS domain names to data. DNS enables us to use hierarchical, friendly names to easily locate computers and other resources on an IP network.

In early days, the job of DNS was done by hosts file, however, it was not scalable. Network administrators entered names and IP addresses into hosts, and computers used the file for name resolution. DNS replaces the Hosts file with a distributed database that implements a hierarchical naming system.

## Domain Namespace
The naming system on which DNS is based is a hierarchical and logical tree structure called the *domain namespace*. Below is a part of internet namespace (private orgs can have their own domain namespace):

<img src="images/domain_namespace.png">

Each node in the above tree is a **DNS name**. A **DNS domain** is a branch under the node, for example, `acme.com` is a DNS domain. DNS domains can
contain both hosts (computers or services) and other domains (referred to as *subdomains*). `legal` is a subdomain created by subdividing the `acme` domain.

### FQDN
Or Fully Qualified Domain Name, is how every node in the DNS domain tree can be identified. For example, `legal.acme.com.` is an FQDN (notice the dot character at the end).

### TLD
Or Top Level Domain, lie beneath the Root of the Internet domain namespace. There are three types of top-level domains:
1. **Organizational domains:** These are named by using a 3-character code that indicates the primary function or activity of the organizations contained within the DNS domain. Generally used by US government departments.
2. **Geographical domains:** These are named by using the 2-character country/region codes.
3. **Reverse domains:** This is a special domain, named in-addr.arpa, that is used for IP address-to-name mappings.

Beneath the top-level domains, an Internet name authority delegates domains to organizations that connect to the Internet. The organizations to which an Internet name authority delegates a portion of the domain namespace are then responsible for naming the computers and network devices within their assigned domain and its subdivisions.

## Zone
Contiguous portion of the DNS namespace. It contains a series of records stored on a DNS server. Each zone is anchored at a specific domain node. However, zones are not domains. A DNS domain is a branch of the namespace, whereas a zone is a portion of the DNS namespace generally stored in a file, and can contain multiple domains.

DNS zone can contain multiple subdomains and multiple zones can exist on the same server. DNS zones are not necessarily physically separated from one another, zones are strictly used for delegating control.

<img src="images/dns_zones.png" >

### Zone File
A zone file is a simple text file that contains the mappings between domain names and IP addresses. Zone files reside in name servers and generally define the resources available under a specific domain, or the place that one can go to get that information. Within a zone file, records are kept. In its simplest form, a record is basically a single mapping between a resource and a name.

The zone’s `$ORIGIN` is a parameter equal to the zone’s highest level of authority by default. So if a zone file is used to configure the “example.com.” domain, the `$ORIGIN` would be set to `example.com.`. 

## DNS Server
Computers that run DNS server programs containing DNS database information about the DNS domain tree structure. A zone is a contiguous portions of the DNS namespace for which the server is authoritative.

DNS servers store information about no zones, one zone, or multiple zones. When a DNS server receives a DNS query, it attempts to locate the requested information by retrieving data from its local zones. If this fails because the server is not authoritative for the DNS domain requested and thus does not have the data for the requested domain, the server can check its cache, communicate with other DNS servers to resolve the request, or refer the client to another DNS server that might know the answer.

DNS servers use a mechanism called round-robin or load sharing, to share and distribute loads for network resources. Round-robin rotates the order of resource record data returned in a query answer in which multiple RRs exist of the same RR type for a queried DNS domain name. Consider the example below:
```
www.acme.com.    IN A    172.16.64.11
www.acme.com.    IN A    172.17.64.22
www.acme.com.    IN A    172.18.64.33
```
A name server configured to perform round-robin rotates the order of the A resource records when answering client requests.

The DNS resolution process can be summarised in the following flow:
<img src="images/dns_resolution.png">

## Record Types
Within the zone file, we can have many different record types such as:

### Origin and TTL
Whenever we have `$ORIGIN` line in a zone file, this is a shortcut that lets us know that any unterminated hostname references following that line should be presumed to end in the argument following `$ORIGIN`.
```
$ORIGIN domain.com.
$TTL 5m
```

`$TTL` means that remote DNS resolvers should only cache records retrieved from this zone for five minutes before requesting them again.

### SOA Record
The Start of Authority, or SOA, record is a mandatory record in all zone files. It must be the first real record in a file (although $ORIGIN or $TTL specifications may appear above). Example:

```
domain.com.  IN SOA   ns1.domain.com. admin.domain.com. (
                          12083           ; serial number
                          3h              ; refresh interval
                          30m             ; retry interval
                          3w              ; expiry period
                          1h              ; negative TTL
                          )
```
The first argument is the hostname the record applies to—in this case, `domain.com.`. If we had set `$ORIGIN` as `com.` earlier, we could have set this first argument as `domain`. This hostname example as "unterminated" because it doesn't end in a dot.

The second argument `IN SOA` sands for internet and start of authority respectively.

The third argument is an FQDN, and it should be the FQDN of the primary name server for the domain itself. The next argument isn't a FQDN at all, rather it's a perverse way of rewriting an email address (since @ in zone file is a reserve character). We'll ignore rest of the paremeters for now.

### NS Record
The NS or “name server” DNS record type is used to specify the authoritative name servers for a domain. It tells DNS resolvers which servers to contact when it's looking for DNS records for that domain name. Each NS record specifies the name of one authoritative name server for a particular DNS zone. Each zone file must have one SOA record and atleast two NS record.
```
        IN  NS     ns1.domain.com.
        IN  NS     ns2.domain.com.
```

Note that we haven't specified the domain name itself on either line—this is because we've inherited it from the SOA record above. We started that line with `domain.com.`. Since we haven't specified another hostname, these new NS records also apply to that hostname by default.

Another note: when we see @ as a hostname in a zone file, that just means we're using the bare $ORIGIN without any further qualifiers.

### A and AAAA Record
Both of these records map a host to an IP address. The “A” record is used to map a host to an IPv4 IP address, while “AAAA” records are used to map a host to an IPv6 address. Format:
```
host     IN      A       IPv4_address
host     IN      AAAA    IPv6_address
```

Since our SOA record called out a primary server at “ns1.domain.com”, we would have to map this to an address to an IP address:
```
ns1      IN      A        111.222.111.222
```

This is equivalent to using the following FQDN:
```
ns1.domain.com.      IN      A      111.222.111.222
```

Other concrete example (always map www):
```
www     IN  A       222.222.222.222
domain.com.     IN  A       222.222.222.222
```

We also have the option of resolving anything that under this domain that is not defined explicitly to this server too. We can do this with the “*” wild card:
```
*       IN  A       222.222.222.222
```

### CNAME Record
CNAME records define an alias for canonical name for your server (one defined by an A or AAAA record). For instance, we could have an A name record defining the “server1” host and then use the “www” as an alias for this host:
```
server1     IN  A       111.111.111.111
www         IN  CNAME   server1
```

### CAA Record
CAA records are used to specify which Certificate Authorities (CAs) are allowed to issue SSL/TLS certificates for your domain. As of September 8, 2017 all CAs are required to check for these records before issuing a certificate. If no record is present, any CA may issue a certificate. Otherwise, only the specified CAs may issue certificates. CAA records can be applied to single hosts, or entire domains.
```
example.com.      IN      CAA      0 issue "letsencrypt.org"
```

### Example
A complete zone file (@ refers to $ORIGIN):
```
$ORIGIN example.com. 
$TTL 86400 
@	IN	SOA	dns1.example.com.	hostmaster.example.com. (
			2001062501 ; serial                     
			21600      ; refresh after 6 hours                     
			3600       ; retry after 1 hour                     
			604800     ; expire after 1 week                     
			86400 )    ; minimum TTL of 1 day  
		     
		           
	IN	NS	dns1.example.com.       
	IN	NS	dns2.example.com.        
	
	
	IN	MX	10	mail.example.com.       
	IN	MX	20	mail2.example.com.        

	
dns1	IN	A	10.0.1.1
dns2	IN	A	10.0.1.2	

			       
server1	IN	A	10.0.1.5        
server2	IN	A	10.0.1.6

       
ftp	IN	A	10.0.1.3
	IN	A	10.0.1.4
	
mail	IN	CNAME	server1
mail2	IN	CNAME	server2


www	IN	CNAME	server1
```

## Terminal Commands
### Linux
`dig` utility is used to query the DNS of a given server and allows to know the answers from the queried domain servers. Lets take the example of a simple use case:
```bash
$ dig linux.com

; <<>> DiG 9.18.1-1ubuntu1.2-Ubuntu <<>> linux.com
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 57188
;; flags: qr rd ad; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 0
;; WARNING: recursion requested but not available

;; QUESTION SECTION:
;linux.com.                     IN      A

;; ANSWER SECTION:
linux.com.              0       IN      A       23.185.0.3

;; Query time: 210 msec
;; SERVER: 172.21.144.1#53(172.21.144.1) (UDP)
;; WHEN: Tue Oct 24 04:48:18 IST 2023
;; MSG SIZE  rcvd: 52
```

The first line outputs the version of the program (9.18.1) and indicates from where to where is the query being launched. The answer section returns the A record stored in the DNS server. This could be or could not be the IP address of the server, because if something uses a DNS firewall or a “façade” server for security purposes or filter, we would see that first. Take the following example:
```bash
$ dig www.goldman.com

; <<>> DiG 9.18.1-1ubuntu1.2-Ubuntu <<>> www.goldman.com
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 61361
;; flags: qr rd ad; QUERY: 1, ANSWER: 4, AUTHORITY: 0, ADDITIONAL: 0
;; WARNING: recursion requested but not available

;; QUESTION SECTION:
;www.goldman.com.               IN      A

;; ANSWER SECTION:
www.goldman.com.        0       IN      CNAME   www.goldman.com.edgekey.net.
www.goldman.com.edgekey.net. 0  IN      CNAME   e247872.x.akamaiedge.net.
e247872.x.akamaiedge.net. 0     IN      A       23.211.60.14
e247872.x.akamaiedge.net. 0     IN      A       23.211.60.26
```

We may also see multiple A records:
```bash
$ dig soundcloud.com

; <<>> DiG 9.18.1-1ubuntu1.2-Ubuntu <<>> soundcloud.com
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 12537
;; flags: qr rd ad; QUERY: 1, ANSWER: 4, AUTHORITY: 0, ADDITIONAL: 0
;; WARNING: recursion requested but not available

;; QUESTION SECTION:
;soundcloud.com.                        IN      A

;; ANSWER SECTION:
soundcloud.com.         0       IN      A       18.67.195.59
soundcloud.com.         0       IN      A       18.67.195.72
soundcloud.com.         0       IN      A       18.67.195.3
soundcloud.com.         0       IN      A       18.67.195.76

;; Query time: 69 msec
;; SERVER: 172.21.144.1#53(172.21.144.1) (UDP)
;; WHEN: Tue Oct 24 04:51:40 IST 2023
;; MSG SIZE  rcvd: 110
```

To use a specific resolving server:
```bash
$ dig soundcloud.com @8.8.8.8
```

To query a specific record type:
```bash
$ dig soundcloud.com -t CAA

; <<>> DiG 9.18.1-1ubuntu1.2-Ubuntu <<>> soundcloud.com -t CAA
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 56671
;; flags: qr rd ad; QUERY: 1, ANSWER: 7, AUTHORITY: 0, ADDITIONAL: 0
;; WARNING: recursion requested but not available

;; QUESTION SECTION:
;soundcloud.com.                        IN      CAA

;; ANSWER SECTION:
soundcloud.com.         0       IN      CAA     0 issue "amazon.com"
soundcloud.com.         0       IN      CAA     0 issue "amazonaws.com"
soundcloud.com.         0       IN      CAA     0 issue "amazontrust.com"
soundcloud.com.         0       IN      CAA     0 issue "awstrust.com"
soundcloud.com.         0       IN      CAA     0 issue "digicert.com"
soundcloud.com.         0       IN      CAA     0 issue "globalsign.com"
soundcloud.com.         0       IN      CAA     0 issue "letsencrypt.org"

;; Query time: 60 msec
;; SERVER: 172.21.144.1#53(172.21.144.1) (UDP)
;; WHEN: Tue Oct 24 04:59:32 IST 2023
;; MSG SIZE  rcvd: 270
```
```bash
$ dig soundcloud.com -t MX

; <<>> DiG 9.18.1-1ubuntu1.2-Ubuntu <<>> soundcloud.com -t MX
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 47155
;; flags: qr rd ad; QUERY: 1, ANSWER: 5, AUTHORITY: 0, ADDITIONAL: 0
;; WARNING: recursion requested but not available

;; QUESTION SECTION:
;soundcloud.com.                        IN      MX

;; ANSWER SECTION:
soundcloud.com.         0       IN      MX      10 aspmx.l.google.com.
soundcloud.com.         0       IN      MX      20 alt1.aspmx.l.google.com.
soundcloud.com.         0       IN      MX      20 alt2.aspmx.l.google.com.
soundcloud.com.         0       IN      MX      50 aspmx2.googlemail.com.
soundcloud.com.         0       IN      MX      50 aspmx3.googlemail.com.

;; Query time: 70 msec
;; SERVER: 172.21.144.1#53(172.21.144.1) (UDP)
;; WHEN: Tue Oct 24 05:00:06 IST 2023
;; MSG SIZE  rcvd: 232
```

## Geo Aware DNS
In this case, the authoritative nameserver determines the geographical location of the resolving name server and returns the IP address of application server closest to the resolving name server.